**bold text**#**PA5**
####Team:

  - Diego Ordonez Pasaje
  - Maria Rossano
  - Tanwi Sarkar


####Name of Data Set:
  - Polycystic ovary syndrome (PCOS) 

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn import neighbors
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive
drive.mount('/content/drive')
from sklearn.model_selection import learning_curve
import plotly.graph_objects as go

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report, plot_confusion_matrix

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.ticker import MaxNLocator
from sklearn import neighbors

#from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model

from IPython.display import Image

**Load Data**

In [ ]:
# Load Original Data
PCOS_path = "../Datasets/"

PCOS_file = PCOS_path + "PCOS_data_without_infertility.xlsx"
PCOS = pd.read_excel(PCOS_file, sheet_name="Full_new")

In [ ]:
PCOS.columns

Index(['Sl. No', 'Patient File No.', 'PCOS (Y/N)', ' Age (yrs)', 'Weight (Kg)',
       'Height(Cm) ', 'BMI', 'Blood Group', 'Pulse rate(bpm) ',
       'RR (breaths/min)', 'Hb(g/dl)', 'Cycle(R/I)', 'Cycle length(days)',
       'Marraige Status (Yrs)', 'Pregnant(Y/N)', 'No. of aborptions',
       '  I   beta-HCG(mIU/mL)', 'II    beta-HCG(mIU/mL)', 'FSH(mIU/mL)',
       'LH(mIU/mL)', 'FSH/LH', 'Hip(inch)', 'Waist(inch)', 'Waist:Hip Ratio',
       'TSH (mIU/L)', 'AMH(ng/mL)', 'PRL(ng/mL)', 'Vit D3 (ng/mL)',
       'PRG(ng/mL)', 'RBS(mg/dl)', 'Weight gain(Y/N)', 'hair growth(Y/N)',
       'Skin darkening (Y/N)', 'Hair loss(Y/N)', 'Pimples(Y/N)',
       'Fast food (Y/N)', 'Reg.Exercise(Y/N)', 'BP _Systolic (mmHg)',
       'BP _Diastolic (mmHg)', 'Follicle No. (L)', 'Follicle No. (R)',
       'Avg. F size (L) (mm)', 'Avg. F size (R) (mm)', 'Endometrium (mm)',
       'Unnamed: 44'],
      dtype='object')

# **0. Data Transformation and Feature Engineering**

In [ ]:
# ----------------------------------------------------------
# ----------------------------------------------------------
## **STEP 1. Drop non-features/labels columns and misaligned column**
# columns to be droped
columns_drop =['Sl. No', 'Patient File No.', \
                'Unnamed: 44']

# labels name
# JUST FOR DATA TRANSFORMATION, VARIABLE WILL BE REDIRECTED LATER:
labels_name ='PRL(ng/mL)' 

# 1. Drop columns in `not_to_be_used`
PCOS.drop(columns_drop, axis=1, inplace=True)

# Check
display(PCOS.head())
display(PCOS.tail())

# ----------------------------------------------------------
# ----------------------------------------------------------
## **STEP 2. Address non float values**
### Feature 'AMH(ng/mL)' has a none float64 value in row: `305`
### Feature 'II    beta-HCG(mIU/mL)' has a none float64 value in row: `123`

# 2.  Address non float values in rows [305, 123].
# A. 
# Rest of values in PCOS.iloc[305] seem to be consistent w/neighbouring rows
#PCOS.iloc[[301, 302, 303, 304, 305]].T

# Thus, I suggest converting their value "a" into a nan
# and treat it as a missing value.
PCOS.replace(PCOS['AMH(ng/mL)'].iloc[305], np.nan, inplace=True)
PCOS['AMH(ng/mL)'] = pd.to_numeric(PCOS['AMH(ng/mL)'], errors = 'coerce')

# B.
# PCOS.iloc[[123, 124, 125]].T
# Seems to be an extra "." in the feature value "1.99.".
# Check with `PCOS['II    beta-HCG(mIU/mL)'].iloc[120:125].T`
PCOS.replace(PCOS['II    beta-HCG(mIU/mL)'].iloc[123], float(1.99), \
             inplace=True)
PCOS['II    beta-HCG(mIU/mL)'] = pd.to_numeric(PCOS['II    beta-HCG(mIU/mL)'],\
                                               errors='coerce')


# Check dtypes
PCOS.dtypes

# Check those specific feature values
print(f"\n305: {PCOS['AMH(ng/mL)'].iloc[305]}")
print(f"123: {PCOS['II    beta-HCG(mIU/mL)'].iloc[123]}\n")

# ----------------------------------------------------------
# ----------------------------------------------------------
## **STEP 3. Drop outliers (i.e., rows with outliers)**

# 3. Drop outliers (i.e, rows with outliers):
rows_drop_dict = {"Pulse":[223, 296],
                  "BP_Systolic": [161], 
                  "BP_Diastolic": [200]}

drops = []
for val in rows_drop_dict.values():
  drops += val

PCOS.drop(drops, axis=0, inplace=True)


# ----------------------------------------------------------
# ----------------------------------------------------------
## **STEP 4. Reindex**
# Reset index of df
PCOS.reset_index(drop=True, inplace=True)

,PCOS (Y/N),Age (yrs),Weight (Kg),Height(Cm),BMI,Blood Group,Pulse rate(bpm),RR (breaths/min),Hb(g/dl),Cycle(R/I),...,Pimples(Y/N),Fast food (Y/N),Reg.Exercise(Y/N),BP _Systolic (mmHg),BP _Diastolic (mmHg),Follicle No. (L),Follicle No. (R),Avg. F size (L) (mm),Avg. F size (R) (mm),Endometrium (mm)
0,0.0,28.0,44.6,152.0,19.300000,15.0,78.0,22.0,10.48,2.0,...,0.0,1.0,0.0,110.0,80.0,3.0,3.0,18.0,18.0,8.5
1,0.0,36.0,65.0,161.5,24.921163,15.0,74.0,20.0,11.70,2.0,...,0.0,0.0,0.0,120.0,70.0,3.0,5.0,15.0,14.0,3.7
2,1.0,33.0,68.8,165.0,25.270891,11.0,72.0,18.0,11.80,2.0,...,1.0,1.0,0.0,120.0,80.0,13.0,15.0,18.0,20.0,10.0
3,0.0,37.0,65.0,148.0,29.674945,13.0,72.0,20.0,12.00,2.0,...,0.0,0.0,0.0,120.0,70.0,2.0,2.0,15.0,14.0,7.5
4,0.0,25.0,52.0,161.0,20.060954,11.0,72.0,18.0,10.00,2.0,...,0.0,0.0,0.0,120.0,80.0,3.0,4.0,16.0,14.0,7.0


,PCOS (Y/N),Age (yrs),Weight (Kg),Height(Cm),BMI,Blood Group,Pulse rate(bpm),RR (breaths/min),Hb(g/dl),Cycle(R/I),...,Pimples(Y/N),Fast food (Y/N),Reg.Exercise(Y/N),BP _Systolic (mmHg),BP _Diastolic (mmHg),Follicle No. (L),Follicle No. (R),Avg. F size (L) (mm),Avg. F size (R) (mm),Endometrium (mm)
536,0.0,35.0,50.0,164.592,18.5,17.0,72.0,16.0,11.0,2.0,...,0.0,0.0,0.0,110.0,70.0,1.0,0.0,17.5,10.0,6.7
537,0.0,30.0,63.2,158.000,25.3,15.0,72.0,18.0,10.8,2.0,...,0.0,0.0,0.0,110.0,70.0,9.0,7.0,19.0,18.0,8.2
538,0.0,36.0,54.0,152.000,23.4,13.0,74.0,20.0,10.8,2.0,...,0.0,0.0,0.0,110.0,80.0,1.0,0.0,18.0,9.0,7.3
539,0.0,27.0,50.0,150.000,22.2,15.0,74.0,20.0,12.0,4.0,...,1.0,0.0,0.0,110.0,70.0,7.0,6.0,18.0,16.0,11.5
540,1.0,23.0,82.0,165.000,30.1,13.0,80.0,20.0,10.2,4.0,...,1.0,1.0,0.0,120.0,70.0,9.0,10.0,19.0,18.0,6.9



305: nan
123: 1.99



## **Feature Enconding**

In [ ]:
# Create feature labels of encoded features
features_categ_encode = ['Blood Group', 'Cycle(R/I)']
features_categ_rename = ['Blood_Group', 'Cycle(R/I)']
features_categ_rename = ["enc_" + feature.lower() for feature in features_categ_rename]


PCOS = pd.get_dummies(PCOS, columns=features_categ_encode, prefix=features_categ_rename)

## **Split Data**

In [ ]:
# Select data
pcos_clean_columns = PCOS.columns
X = PCOS[pcos_clean_columns[1:]]
y = PCOS[labels_name]

# Split data in training, validation and testing

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=23)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=23)
# y_train.head()

In [ ]:
y

0      45.16
1      20.09
2      10.52
3      36.90
4      30.09
       ...  
532     5.30
533    21.09
534    96.41
535    38.89
536    20.74
Name: PRL(ng/mL), Length: 537, dtype: float64

## **Feature transfomartion across rows and columns**

In [ ]:
X_data_list = [X_train,X_val, X_test]
# ----------------------------------------------------------
# ----------------------------------------------------------
## **STEP 5. filling `Nan`'s values for specific rows**
# 305, 458, 156

# A. PCOS.iloc[[ 458]].T has a Nan value for the 'Marraige Status (Yrs)'
# This woman (PCOS.iloc[[ 458]]) has 36 year old
# So, we calculate the mean of 'Marraige Status (Yrs)' for those that only have 36 year old
# Then, we use this mean to fill the Nan value: 

# data points of women with 36 year all and their respective years of marriege:

for X_data in X_data_list:
  age36Marraige = X_data[X_data[' Age (yrs)']==36]['Marraige Status (Yrs)']
  age36Marraige_mean = age36Marraige.mean() 
  X_data['Marraige Status (Yrs)'].fillna(age36Marraige_mean, inplace=True)

  # B. PCOS.iloc[[ 156]].T has a Nan value for the 'Fast food (Y/N)'
  # We fill this value with the mode of 'Fast food (Y/N)':

  X_data['Fast food (Y/N)'].fillna(X_data['Fast food (Y/N)'].mode()[0], inplace=True)

  # C. PCOS.iloc[[ 305]].T has a Nan value for the feature "AMH(ng/mL)"
  mean_amh = X_data["AMH(ng/mL)"].mean()
  X_data["AMH(ng/mL)"].fillna(mean_amh, inplace=True)



  # ----------------------------------------------------------
  # ----------------------------------------------------------
  ## **STEP 6. Recalculate BMI and drop `BMI` column**
  # Recalculate BMI
  def get_bmi(row):
    w = row["Weight (Kg)"]
    h = float(row["Height(Cm) "])/100
    bmi = w/h**2
    return bmi

  # Create a column with the recalculated BMI
  X_data["bmi_recalc"] = X_data.apply(get_bmi, axis=1)

  # Drop column
  X_data.drop("BMI", axis=1, inplace=True)

  # ----------------------------------------------------------
  # ----------------------------------------------------------
  ## **STEP 7. Aggregate Follicle L and R, i.e., get total number of follicles**
  def get_TotalFollicles(row):
    r = row['Follicle No. (L)']
    l = row['Follicle No. (R)']
    tf = r+l 
    return tf
    
  X_data['Total No of Folicles'] = X_data.apply(get_TotalFollicles, axis=1)
  X_data.drop(['Follicle No. (R)', 'Follicle No. (L)'], axis=1, inplace=True)

  # ----------------------------------------------------------
  # ----------------------------------------------------------
  ## **STEP 8. Aggregate Avg. F size (L) and (R), i.e., get average follicles' size**

  def get_TotalAverageFollicles(row):
    ra = row['Avg. F size (L) (mm)']
    la = row['Avg. F size (R) (mm)']
    tfa = (ra+la)/2
    return tfa
    
  X_data['Average size of Folicles (mm)'] = \
  X_data.apply(get_TotalAverageFollicles, axis=1)

  X_data.drop(['Avg. F size (L) (mm)', 'Avg. F size (R) (mm)'], \
            axis=1, inplace=True)

  # ----------------------------------------------------------
  # ----------------------------------------------------------
  ## **STEP 9. Get Mean Arterial Pressure
  ## get average follicles' size**

  def get_MeanArterialPreassure(row):
    d = row['BP _Diastolic (mmHg)']
    s = row['BP _Systolic (mmHg)']
    map = (2*d+s)/3 
    return map
    
  X_data["Mean_Arterial_Preassure"] = X_data.apply(get_MeanArterialPreassure, axis=1)



  # check columns

  print(f"Total of features + label: {len(X_data.columns)}")

Total of features + label: 49
Total of features + label: 49
Total of features + label: 49


In [ ]:
# Final check
for X_data in X_data_list:
  print(f"Total of null values: {X_data.isnull().sum().sum()}\n")
  print(f"dtypes for all columns:\n{X_data.dtypes}")

Total of null values: 0

dtypes for all columns:
 Age (yrs)                       float64
Weight (Kg)                      float64
Height(Cm)                       float64
Pulse rate(bpm)                  float64
RR (breaths/min)                 float64
Hb(g/dl)                         float64
Cycle length(days)               float64
Marraige Status (Yrs)            float64
Pregnant(Y/N)                    float64
No. of aborptions                float64
  I   beta-HCG(mIU/mL)           float64
II    beta-HCG(mIU/mL)           float64
FSH(mIU/mL)                      float64
LH(mIU/mL)                       float64
FSH/LH                           float64
Hip(inch)                        float64
Waist(inch)                      float64
Waist:Hip Ratio                  float64
TSH (mIU/L)                      float64
AMH(ng/mL)                       float64
PRL(ng/mL)                       float64
Vit D3 (ng/mL)                   float64
PRG(ng/mL)                       float64
RBS(mg/d

In [ ]:
# Split feature names into two groups, categorical and numerical

columns_names = X_train.columns # either X_train or X_test is fine

features_categ = []
features_numeric = []

for column in columns_names:
  unq = X_train[column].unique()
  if column != labels_name:
    if len(unq) <= 3 or (column == "Blood Group"):
      features_categ.append(column)
    else:
      features_numeric.append(column)


features_all = features_categ + features_numeric


In [ ]:
# Display features names
display(features_categ)
display(features_numeric)
print(f"Features total: {len(features_all)}")

['Pregnant(Y/N)',
 'Weight gain(Y/N)',
 'hair growth(Y/N)',
 'Skin darkening (Y/N)',
 'Hair loss(Y/N)',
 'Pimples(Y/N)',
 'Fast food (Y/N)',
 'Reg.Exercise(Y/N)',
 'BP _Diastolic (mmHg)',
 'enc_blood_group_11.0',
 'enc_blood_group_12.0',
 'enc_blood_group_13.0',
 'enc_blood_group_14.0',
 'enc_blood_group_15.0',
 'enc_blood_group_16.0',
 'enc_blood_group_17.0',
 'enc_blood_group_18.0',
 'enc_cycle(r/i)_2.0',
 'enc_cycle(r/i)_4.0',
 'enc_cycle(r/i)_5.0']

[' Age (yrs)',
 'Weight (Kg)',
 'Height(Cm) ',
 'Pulse rate(bpm) ',
 'RR (breaths/min)',
 'Hb(g/dl)',
 'Cycle length(days)',
 'Marraige Status (Yrs)',
 'No. of aborptions',
 '  I   beta-HCG(mIU/mL)',
 'II    beta-HCG(mIU/mL)',
 'FSH(mIU/mL)',
 'LH(mIU/mL)',
 'FSH/LH',
 'Hip(inch)',
 'Waist(inch)',
 'Waist:Hip Ratio',
 'TSH (mIU/L)',
 'AMH(ng/mL)',
 'Vit D3 (ng/mL)',
 'PRG(ng/mL)',
 'RBS(mg/dl)',
 'BP _Systolic (mmHg)',
 'Endometrium (mm)',
 'bmi_recalc',
 'Total No of Folicles',
 'Average size of Folicles (mm)',
 'Mean_Arterial_Preassure']

Features total: 48


## **Scale Numerical Features**

In [ ]:
# Apply min-max scaling

X_train_minmax, X_val_minmax, X_test_minmax = X_train.copy(), X_val.copy(), X_test.copy()
#X_train_standard, X_test_standard = X_train.copy(), X_test.copy()

def get_minmax_scaled(df, feature, scaler_type):
  scaler = scaler_type()
  feature_scaled = scaler.fit_transform(df[[feature]].values)
  feature_scaled = (2 * feature_scaled) - 1
  df[feature] = feature_scaled
  return df

for feature in features_numeric:
  get_minmax_scaled(X_train_minmax, feature, MinMaxScaler)
  get_minmax_scaled(X_val_minmax, feature, MinMaxScaler)
  get_minmax_scaled(X_test_minmax, feature, MinMaxScaler)

In [ ]:
def print_results(results):
    print('Best Params: {}\n'.format(results.best_params_))

## **1. Implement a linear surface for dataset**

In [ ]:
reg = LinearRegression().fit(X_train_minmax, y_train)
#yhat_test  = reg.predict(X_test_minmax)
#print('Predictions:', yhat_test)
#print("Score:", reg.score(X_train_minmax, y_train))
#print("Coef:", reg.coef_)
#print('Intercept:\n', reg.intercept_)

##**2-3. Implement linear least squares with L2 regularization (tweak the value of alpha) - RIDGE**

In [ ]:
L2r_alphaDefault = Ridge().fit(X_train_minmax, y_train)
# alphas before: 1,2,4
L2r_alpha005 = Ridge(alpha=.005).fit(X_train_minmax, y_train)
L2r_alpha05 = Ridge(alpha=.05).fit(X_train_minmax, y_train)
L2r_alpha5 = Ridge(alpha=.5).fit(X_train_minmax, y_train)

##**4-5. Implement linear least squares with L1 regularization (tweak the value of alpha)- LASSO**

In [ ]:
L1r_alphaDefault = linear_model.Lasso().fit(X_train_minmax, y_train)
L1r_alpha005 = linear_model.Lasso(alpha=0.005).fit(X_train_minmax, y_train)
L1r_alpha05 = linear_model.Lasso(alpha=0.05).fit(X_train_minmax, y_train)
L1r_alpha5 = linear_model.Lasso(alpha=.5).fit(X_train_minmax, y_train)

## **ML Model Comparison**

In [ ]:
#Define models:

LinearReg = reg 
Ridge_alphaDefault = L2r_alphaDefault
Ridge_alpha005 = L2r_alpha005
Ridge_alpha05 = L2r_alpha05
Ridge_alpha5 = L2r_alpha5
Lasso_alphaDefault = L1r_alphaDefault
Lasso_alpha005 = L1r_alpha005
Lasso_alpha05 = L1r_alpha05
Lasso_alpha5 = L1r_alpha5

Models_list = [LinearReg, 
               Ridge_alphaDefault,
               Ridge_alpha005,
               Ridge_alpha05,
               Ridge_alpha5,
               Lasso_alphaDefault,
               Lasso_alpha005,
               Lasso_alpha05,
               Lasso_alpha5]

In [ ]:
#Create a defenition to evaluate models:
scores_list = []
def evaluate_model(model):
  yhat_test  = model.predict(X_test_minmax)
  mae = '{:.2e}'.format(float(mean_absolute_error(y_test, yhat_test)))
  mse = '{:.2e}'.format(float(mean_squared_error(y_test,yhat_test)))
  rmse = '{:.2e}'.format(float(np.sqrt(mean_squared_error(y_test, yhat_test))))
  r2 = '{:.2e}'.format(float(r2_score(y_test,yhat_test)))
  scores_list.append({
                      'Mean Absolute Error': mae,
                      'Mean Squared Error': mse,
                      'Root Mean Squared Error': rmse,
                      'Coefficient of determination': r2,
                      })

In [ ]:
#for loop to evaluate all models:
for model in Models_list:
  evaluate_model(model)

summary_models = pd.DataFrame(scores_list)
# summary_models

In [ ]:
model_name = ['LinearRegression', 
               'Ridge_alphaDefault',
               'Ridge_alpha005',
               'Ridge_alpha05',
               'Ridge_alpha5',
               'Lasso_alphaDefault',
               'Lasso_alpha005',
               'Lasso_alpha05',
               'Lasso_alpha5']

summary_models['Model'] = model_name
summary_models

,Mean Absolute Error,Mean Squared Error,Root Mean Squared Error,Coefficient of determination,Model
0,3.26e-15,2.00e-29,4.47e-15,1.00e+00,LinearRegression
1,1.85e-04,7.16e-08,2.67e-04,1.00e+00,Ridge_alphaDefault
2,9.53e-07,1.85e-12,1.36e-06,1.00e+00,Ridge_alpha005
3,9.39e-06,1.82e-10,1.35e-05,1.00e+00,Ridge_alpha05
4,9.28e-05,1.80e-08,1.34e-04,1.00e+00,Ridge_alpha5
5,5.03e-02,5.24e-03,7.24e-02,1.00e+00,Lasso_alphaDefault
6,2.51e-04,1.31e-07,3.62e-04,1.00e+00,Lasso_alpha005
7,2.51e-03,1.31e-05,3.62e-03,1.00e+00,Lasso_alpha05
8,2.51e-02,1.31e-03,3.62e-02,1.00e+00,Lasso_alpha5


### **Business case**: prediction of the concentration of prolactin (PRL) using non-invase clinical methods. 
Determining the concentration of PRL in women can aid in the prediction of pregancy or diseases like tumors in the pituitary gland. 

### **Discussion of results**

- Overall, the introduction of a regularizer into our linear regression problem results in an improvement of the accuracy of prediction of tour labels.
- The overall best performance, i.e., lowest MAE,  is obtained when a Lasso (L1) regularizer is used.
- Overall, the MSE among all the models evaluated here does not vary dramatically. Which can be interpreted as a consistent error $y-\hat{y}$ among predicted values. 
- However, in all cases, the prediction of the observed values is poor, which is reflected by negative and small ($< 0.5$) $r^{2}$ scores. 
- In the case of our dataset, w/48 features, where some of them might not be big contibutors in the prediction of the label "PRL", using a Lasso regularizer is a better approach than using a Ridge regularizer. 
- It might be that the business case defined around the concentration of PRL is not relevant for this pool of features. From the correlation matrix obtained during the EDA stage, we found that there is no significant correlation between the PRL concentration and the rest of the features.
